# Script to Create Figure 3 - MUR Temperature, ERA5 Winds, and Percent Footrpint for each EBUS/Subregion

Reference to multiple y-axis plots:
https://matplotlib.org/3.4.3/gallery/ticks_and_spines/multiple_yaxis_with_spines.html

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import math
import glob
import warnings 
warnings.simplefilter('ignore') 

In [ ]:
#function to equally calculate ticks
def calculate_ticks(ax, ticks, round_to=0.1, center=False):
    upperbound = np.ceil(ax.get_ybound()[1]/round_to)
    lowerbound = np.floor(ax.get_ybound()[0]/round_to)
    dy = upperbound - lowerbound
    fit = np.floor(dy/(ticks - 1)) + 1
    dy_new = (ticks - 1)*fit
    if center:
        offset = np.floor((dy_new - dy)/2)
        lowerbound = lowerbound - offset
    values = np.linspace(lowerbound, lowerbound + dy_new, ticks)
    return values*round_to

### Calculate MUR Nearshore Monthly Average

In [ ]:
#create a list of EBUS to loop through
ebus_list = ['Humboldt', 'Iberian-Canary', 'California', 'Benguela']

#create an empty dataframe
nearshore_means = pd.DataFrame()

#loop through each EBUS
for ebus in ebus_list:
    
    #select out coordinates based on EBUS
    if ebus == 'California':
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
        lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
        sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (modified subregions)
        #sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (original subregions)
        #sst_upw = [17.337725, 14.049598, 13.024406] #long term, full region means
        reg = 3
    elif ebus == 'Humboldt':
        lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
        lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
        sst_upw = [14.118310928344727, 18.504968643188477, 18.300222396850586] #nearshore means (modified supregions)
        #sst_upw = [14.118310928344727, 19.07120704650879, 18.2731876373291] #nearshore means (original subregions)
        #sst_upw = [14.972103, 19.707349, 19.989355] #long term, full region means
        reg = 3
    elif ebus == 'Iberian-Canary':
        lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
        lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
        sst_upw = [22.11833381652832, 18.946455001831055, 16.212278366088867] #nearshore means (modified subregions with central Iberian changed)
        #sst_upw = [22.574838638305664, 19.255435943603516, 16.212278366088867] #nearshore means (original subregions)
        #sst_upw = [23.338499, 20.541109, 17.248304] #long term, full region means
        reg = 3
    else: #Benguela
        lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
        lons = np.array([[13, 20],[10, 17],[8, 15]])
        sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)
        #sst_upw = [15.341087341308594, 18.181407928466797] #nearshore means (original subregions)
        #sst_upw = [17.249544, 19.3302917] #long term, full region means
        reg = 3
    
    #open the monthly means file
    ds = xr.open_dataset('../../data/data_download/Offshore_Onshorebands/'+ebus+'_NearshoreMonthlyMean_30km.nc')
    ds.close()
    
    #loop through the number of regions
    for region in range(reg):
        
        #select out the region, groupby the month, take the mean, and convert it to a dataframe
        df = ds.sel(lat=slice(lats[region,0],lats[region,1]),lon=slice(lons[region,0],lons[region,1])).groupby(ds['time'].dt.month).mean(...).to_dataframe()
        
        #add a column for the region
        df['EBUS'] = ebus
        df['Region'] = str(region)
        
        nearshore_means = pd.concat([nearshore_means, df])

nearshore_means.loc[(nearshore_means['EBUS'].isin(['Iberian-Canary', 'California'])) & (nearshore_means['Region'] == '0'), 'Region'] = 'Equatorward'
nearshore_means.loc[(nearshore_means['EBUS'].isin(['Iberian-Canary', 'California'])) & (nearshore_means['Region'] == '1'), 'Region'] = 'Central'
nearshore_means.loc[(nearshore_means['EBUS'].isin(['Iberian-Canary', 'California'])) & (nearshore_means['Region'] == '2'), 'Region'] = 'Poleward'

nearshore_means.loc[(nearshore_means['EBUS'].isin(['Benguela', 'Humboldt'])) & (nearshore_means['Region'] == '0'), 'Region'] = 'Poleward'
nearshore_means.loc[(nearshore_means['EBUS'].isin(['Humboldt'])) & (nearshore_means['Region'] == '1'), 'Region'] = 'Central'
nearshore_means.loc[(nearshore_means['EBUS'].isin(['Benguela'])) & (nearshore_means['Region'] == '1'), 'Region'] = 'Central'
nearshore_means.loc[(nearshore_means['EBUS'].isin(['Humboldt'])) & (nearshore_means['Region'] == '2'), 'Region'] = 'Equatorward'
nearshore_means.loc[(nearshore_means['EBUS'].isin(['Benguela'])) & (nearshore_means['Region'] == '2'), 'Region'] = 'Equatorward'

nearshore_means.reset_index().to_csv('../../data/MUR_nearshore_monthly_climatology_Modified-Subregions.csv', index = False)
nearshore_means

### Calculate MUR Full-Region Monthly Average

In [ ]:
ebus_list = ['Humboldt', 'Iberian-Canary', 'California', 'Benguela']

for ebus in ebus_list:
    ds = xr.open_dataset('../../data/data_download/Offshore_Onshorebands/'+ebus+'_OffshoreMonthlyClim_30km.nc')
    ds.close()

    ds = ds.groupby(ds.time.dt.month).mean()
    ds.to_netcdf('../../data/data_download/Offshore_Onshorebands/'+ebus+'_OffshoreMonthlyMean_30km.nc')
    
ds

In [ ]:
#create a list of EBUS to loop through
ebus_list = ['Humboldt', 'Iberian-Canary', 'California', 'Benguela']

#create an empty dataframe
full_means = pd.DataFrame()

#loop through each EBUS
for ebus in ebus_list:
    
    #select out coordinates based on EBUS
    if ebus == 'California':
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
        lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
        sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (modified subregions)
        #sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (original subregions)
        #sst_upw = [17.337725, 14.049598, 13.024406] #long term, full region means
        reg = 3
    elif ebus == 'Humboldt':
        lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
        lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
        sst_upw = [14.118310928344727, 18.504968643188477, 18.300222396850586] #nearshore means (modified supregions)
        #sst_upw = [14.118310928344727, 19.07120704650879, 18.2731876373291] #nearshore means (original subregions)
        #sst_upw = [14.972103, 19.707349, 19.989355] #long term, full region means
        reg = 3
    elif ebus == 'Iberian-Canary':
        lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
        lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
        sst_upw = [22.11833381652832, 19.076478958129883, 16.212278366088867] #nearshore means (modified subregions)
        #sst_upw = [22.574838638305664, 19.255435943603516, 16.212278366088867] #nearshore means (original subregions)
        #sst_upw = [23.338499, 20.541109, 17.248304] #long term, full region means
        reg = 3
    else: #Benguela
        lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
        lons = np.array([[13, 20],[10, 17],[8, 15]])
        sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)
        #sst_upw = [15.341087341308594, 18.181407928466797] #nearshore means (original subregions)
        #sst_upw = [17.249544, 19.3302917] #long term, full region means
        reg = 3
    
    #open the monthly means file
    ds = xr.open_dataset('../../data/data_download/Offshore_Onshorebands/'+ebus+'_OffshoreMonthlyMean_30km.nc')
    ds.close()
    
    #loop through the number of regions
    for region in range(reg):
        
        #select out the region, groupby the month, take the mean, and convert it to a dataframe
        df = ds.sel(lat=slice(lats[region,0],lats[region,1]),lon=slice(lons[region,0],lons[region,1])).groupby('month').mean(...).to_dataframe()
        
        #add a column for the region
        df['EBUS'] = ebus
        df['Region'] = str(region)
        
        full_means = pd.concat([full_means, df])

full_means.loc[(full_means['EBUS'].isin(['Iberian-Canary', 'California'])) & (full_means['Region'] == '0'), 'Region'] = 'Equatorward'
full_means.loc[(full_means['EBUS'].isin(['Iberian-Canary', 'California'])) & (full_means['Region'] == '1'), 'Region'] = 'Central'
full_means.loc[(full_means['EBUS'].isin(['Iberian-Canary', 'California'])) & (full_means['Region'] == '2'), 'Region'] = 'Poleward'

full_means.loc[(full_means['EBUS'].isin(['Benguela', 'Humboldt'])) & (full_means['Region'] == '0'), 'Region'] = 'Poleward'
full_means.loc[(full_means['EBUS'].isin(['Humboldt'])) & (full_means['Region'] == '1'), 'Region'] = 'Central'
full_means.loc[(full_means['EBUS'].isin(['Benguela'])) & (full_means['Region'] == '1'), 'Region'] = 'Central'
full_means.loc[(full_means['EBUS'].isin(['Humboldt'])) & (full_means['Region'] == '2'), 'Region'] = 'Equatorward'
full_means.loc[(full_means['EBUS'].isin(['Benguela'])) & (full_means['Region'] == '2'), 'Region'] = 'Equatorward'


full_means.reset_index().to_csv('../../data/MUR_offshore_monthly_climatology_Modified-Subregions.csv', index = False)
full_means

### Calculate ERA5 Winds Monthly Average

In [ ]:
#create a list of EBUS to loop through
ebus_list = ['Humboldt', 'Iberian-Canary', 'California', 'Benguela']

#create an empty dataframe
winds = pd.DataFrame()

#loop through each EBUS
for ebus in ebus_list:
    
    #select out coordinates based on EBUS
    if ebus == 'California':
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
        lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
        sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (modified subregions)
        #sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (original subregions)
        #sst_upw = [17.337725, 14.049598, 13.024406] #long term, full region means
        reg = 3
    elif ebus == 'Humboldt':
        lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
        lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
        sst_upw = [14.118310928344727, 18.504968643188477, 18.300222396850586] #nearshore means (modified supregions)
        #sst_upw = [14.118310928344727, 19.07120704650879, 18.2731876373291] #nearshore means (original subregions)
        #sst_upw = [14.972103, 19.707349, 19.989355] #long term, full region means
        reg = 3
    elif ebus == 'Iberian-Canary':
        lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
        lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
        sst_upw = [22.11833381652832, 19.076478958129883, 16.212278366088867] #nearshore means (modified subregions)
        #sst_upw = [22.574838638305664, 19.255435943603516, 16.212278366088867] #nearshore means (original subregions)
        #sst_upw = [23.338499, 20.541109, 17.248304] #long term, full region means
        reg = 3
    else: #Benguela
        lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
        lons = np.array([[13, 20],[10, 17],[8, 15]])
        sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)
        #sst_upw = [15.341087341308594, 18.181407928466797] #nearshore means (original subregions)
        #sst_upw = [17.249544, 19.3302917] #long term, full region means
        reg = 3
    
    #open the monthly means file
    ds = xr.open_dataset('../../data/data_download/'+ebus+'_ERA5.nc')
    ds.close()
    
    if ebus != 'Benguela':
        #convert the longitude to be -180 to 180
        ds['lon'] = ds['lon'] - 360
    
    #loop through the number of regions
    for region in range(reg):
        
        #select out the region, groupby the month, take the mean, and convert it to a dataframe
        df = ds.sel(lat=slice(lats[region,1],lats[region,0]),lon=slice(lons[region,0],lons[region,1])).groupby(ds['time0'].dt.month).mean(...).to_dataframe()
        
        #add a column for the region
        df['EBUS'] = ebus
        df['Region'] = str(region)
        
        winds = pd.concat([winds, df])

winds.loc[(winds['EBUS'].isin(['Iberian-Canary', 'California'])) & (winds['Region'] == '0'), 'Region'] = 'Equatorward'
winds.loc[(winds['EBUS'].isin(['Iberian-Canary', 'California'])) & (winds['Region'] == '1'), 'Region'] = 'Central'
winds.loc[(winds['EBUS'].isin(['Iberian-Canary', 'California'])) & (winds['Region'] == '2'), 'Region'] = 'Poleward'

winds.loc[(winds['EBUS'].isin(['Benguela', 'Humboldt'])) & (winds['Region'] == '0'), 'Region'] = 'Poleward'
winds.loc[(winds['EBUS'].isin(['Humboldt'])) & (winds['Region'] == '1'), 'Region'] = 'Central'
winds.loc[(winds['EBUS'].isin(['Benguela'])) & (winds['Region'] == '1'), 'Region'] = 'Central'
winds.loc[(winds['EBUS'].isin(['Humboldt'])) & (winds['Region'] == '2'), 'Region'] = 'Equatorward'
winds.loc[(winds['EBUS'].isin(['Benguela'])) & (winds['Region'] == '2'), 'Region'] = 'Equatorward'


winds.reset_index().to_csv('../../data/ERA5_winds_monthly_climatology_Modified-Subregions.csv', index = False)
winds

### Create Figure

In [ ]:
#read in and prepare data
nearshore = pd.read_csv('../../data/MUR_nearshore_monthly_climatology_Modified-Subregions_new.csv')
nearshore = nearshore.rename({'analysed_sst':'nearshore_sst'}, axis = 1)

full = pd.read_csv('../../data/MUR_offshore_monthly_climatology_Modified-Subregions.csv')
full = full.rename({'analysed_sst':'offshore_sst'}, axis = 1)

wind = pd.read_csv('../../data/ERA5_winds_monthly_climatology_Modified-Subregions.csv')
wind.loc[wind['EBUS'].isin(['California', 'Iberian-Canary']), 'northward_wind_at_10_metres'] = wind.loc[wind['EBUS'].isin(['California', 'Iberian-Canary']), 'northward_wind_at_10_metres'] * -1

percent = pd.read_csv('../../data/upwellingfootprint_monthly_nearshore_updated_Modified-Subregions.csv')
percent = percent.groupby(['Month', 'EBUS_name', 'Region_name']).mean(...).reset_index()
percent = percent[['Month', 'EBUS_name', 'Region_name', 'Percent']]
percent = percent.rename({'EBUS_name':'EBUS', 'Region_name':'Region', 'Month':'month'}, axis = 1)

#combine all data together
combined = pd.merge(pd.merge(pd.merge(nearshore, full, on = ['EBUS', 'Region', 'month']), wind, on = ['EBUS', 'Region', 'month']), percent, on = ['EBUS', 'Region', 'month'])

#add a small amount to clean up axes
combined.loc[(combined['EBUS'] == 'California') & (combined['Region'] == 'Poleward'), ['northward_wind_at_10_metres']] = combined.loc[(combined['EBUS'] == 'California') & (combined['Region'] == 'Poleward'), ['northward_wind_at_10_metres']] + 0.6
combined.loc[(combined['EBUS'] == 'California') & (combined['Region'] == 'Poleward'), ['nearshore_sst', 'offshore_sst']] = combined.loc[(combined['EBUS'] == 'California') & (combined['Region'] == 'Poleward'), ['nearshore_sst', 'offshore_sst']] + 0.3

#replace the - with / in Iberian/Canary
combined['EBUS'] = combined['EBUS'].str.replace('-','/')
combined

##### Figure with SST, Wind, and Percent (Vertical)

In [ ]:
#create a 3x4 figure 
fig, axs = plt.subplots(4, 3, figsize=(16, 12))

#create a list of figure labels to loop through
figure_label_list = [['A', 'B', 'C'], ['D', 'E', 'F'], ['G', 'H', 'I'], ['J', 'K', 'L']]

#loop through all of the columns/ebus
for i, fig_legend, ebus in zip([*range(4)], range(4), ['California', 'Iberian/Canary', 'Humboldt', 'Benguela']):
    
    #loop through all of the rows/regions
    for j, fig_label, region in zip([*range(3)], figure_label_list[fig_legend], ['Poleward', 'Central', 'Equatorward']):
        
        #filter the data
        temp = combined[(combined['EBUS'] == ebus) & (combined['Region'] == region)]

        #if this is the last subplot
        if (i == 3) & (j == 2):
            #add a label to that figure (for the legend)
            axs[i, j].plot(temp['month'], temp['northward_wind_at_10_metres'], 'd:', linewidth = 2, markersize = 8, label = 'Wind Speed', c='tab:red')
        #if it's any other subplot besides the last one
        else:
            #plot the data but don't add the label
            axs[i, j].plot(temp['month'], temp['northward_wind_at_10_metres'], 'd:', linewidth = 2, markersize = 8, c='tab:red')
            
        #customize the x ticks to only show four months and label them as months
        axs[i, j].set_xticks([*range(1, 13, 3)])
        axs[i, j].set_xticklabels(['Jan', 'Apr', 'Jul','Oct'])
        
        #add a grid to the data
        #axs[i, j].grid(True, alpha=0.5, zorder=0)
        
        #remove the axis titles for individual plots
        axs[i, j].set_xlabel('')
        axs[i, j].set_ylabel('')
        
        #add a title that represents the region for each plot
        axs[i, j].set_title('(' + fig_label + ') ' + region, fontsize = 15, loc='left', pad = 8)

        #add a second axis
        ax2 = axs[i, j].twinx()
        ax3 = axs[i, j].twinx()

        # Offset the right spine of twin2.  The ticks and label have already been placed on the right by twinx above.
        ax3.spines.right.set_position(("axes", 1.15))

        #if it's the last subplot
        if (i == 3) & (j == 2):
            #add the temperature data with labels so there aren't duplicates in the legend
            ax2.plot(temp['month'], temp['nearshore_sst'], 'o-', c='tab:blue', alpha=0.6, linewidth = 2, label = 'Nearshore SST')
            #ax2.plot(temp['month'], temp['offshore_sst'], 'o-', c='navy', alpha=0.6, linewidth = 2, label = 'Offshore SST')
            ax3.plot(temp['month'], temp['Percent'], '^--', c = 'k', alpha = 0.7, markersize = 7, linewidth = 2, label = 'SST Footprint')
        #if it's any other plot besides the last subplot
        else: 
            #add the temperature data without a label
            ax2.plot(temp['month'], temp['nearshore_sst'], 'o-', c='tab:blue', alpha=0.6, linewidth = 2)
            #ax2.plot(temp['month'], temp['offshore_sst'], 'o-', c='navy', alpha=0.6, linewidth = 2)
            ax3.plot(temp['month'], temp['Percent'], '^--', c = 'k', markersize = 7, alpha = 0.7, linewidth = 2)


        #set the axis labels to be blank
        ax2.set_ylabel('')
        ax2.set_xlabel('')
        ax3.set_ylabel('')
        ax3.set_xlabel('')

        #calculate the ticks so that they're even for the two axes
        axs[i, j].set_yticks([*range(-2, 9, 2)])
        ax2.set_yticks([*range(10, 31, (30-10)//5)])
        ax3.set_yticks([*range(0, 101)])

        #change the color of the axis tick labels to match the lines
        ax2.set_yticklabels([*range(10, 31, (30-10)//5)], c = 'tab:blue')
        axs[i, j].set_yticklabels([*range(-2, 9, 2)], c = 'tab:red')
        ax3.set_yticks([*range(0, 101, 20)])

        #if the wind axis crosses 0
        if (np.min(calculate_ticks(axs[i, j], 6)) <= 0) & (np.max(calculate_ticks(axs[i, j], 6)) >= 0):
            #add a horizontal line for 0 matching the color of the wind
            axs[i, j].axhline(c = 'tab:red', linewidth = 2, alpha=0.8)
            
        #set plot width
        [x.set_linewidth(2) for x in axs[i, j].spines.values()]
        [x.set_linewidth(2) for x in ax3.spines.values()]
        
        #set tick width
        axs[i, j].tick_params('both', labelsize=14, width = 2, length = 6)
        ax2.tick_params('both', labelsize=14, width = 2, length = 6)
        ax3.tick_params('both', labelsize=14, width = 2, length = 6)



#add a legend to the figure
fig.legend(loc='center left', bbox_to_anchor=(1.05, 0.45), prop={'size': 15})
#adjust the figure
fig.tight_layout() 
fig.subplots_adjust(top = 0.9, bottom=0.01, hspace=0.6, wspace=0.4)

#add text elements for the axis labels
fig.text(-0.015, 0.35, 'Wind Speed (m/s)', c='tab:red', fontsize = 24, rotation = 90)
#fig.text(0.48, -0.05, 'Month', fontsize = 24, ha='center')
fig.text(1.01, 0.35, 'Temperature (°C)', c='tab:blue', fontsize = 24, rotation = 270, ha = 'center')
fig.text(1.04, 0.35, 'SST footprint (%)', c='k', fontsize = 24, rotation = 270, ha = 'center')

#add text elements for the ebus 
fig.text(0.025, 0.935, 'California', fontsize = 21, ha='left')
fig.text(0.025, 0.6855, 'Iberian/Canary', fontsize = 21, ha='left')
fig.text(0.025, 0.445, 'Humboldt', fontsize = 21, ha='left')
fig.text(0.025, 0.1955, 'Benguela', fontsize = 21, ha='left')

# #shift the right Benguela plot closer towards center to minimize the white space
# box = axs[3, 2].get_position()
# box.x0 = box.x0 - 0.332
# box.x1 = box.x1 - 0.332
# axs[3, 2].set_position(box)

#fig.suptitle('Testing', fontsize = 24)
plt.savefig('../../figures/Figure3_MURSST_ERA5Wind_Percent_vertical.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
